In [857]:
##### imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', 1000)

In [858]:
##### read in college data as json file, drop unnecessary columns
with open('/Users/kstern/ds/metis/Kevin/Projects/Luther/data/collegeData.json') as f:
   data = json.load(f)
df = pd.DataFrame(data)
df = df.drop(['retry_times','id', 'download_slot', 'download_timeout', 'depth', 'download_latency'], axis = 1)

In [859]:
df.shape

(1975, 42)

### SAMPLE DATA

{'nearestAirport': u'20 mile(s) from campus in Lexington', 'freshmanHousingGuaranteed': u'Freshmen are guaranteed housing', 'pctWithLoans': u'70%', 'numGrad': u'2,517', 'SATMath': u'483 average ', 'ACT': u'22 average ', 'classSize': u'2-9 students: 20% of classes', 'pctInternationalStudents': u'1.6% from 44 countries', 'nearestTrain': u'\xa0', 'id': u'944', 'totalCost': [u'In-state: $22,014 ', u' Out-of-state: $31,792'], 'tuitionCost': [u'In-state: $9,296', u'Out-of-state: $19,074'], 'roomCost': [u'$9,018'], 'schoolColors': u'Maroon and white', 'sportsDivision': u'NCAA Division I', 'sororities': u'6% of women participate', 'averageSalary': u'Not reported', 'avgGPA': u'3.29', 'pctStudentsGraduatingin4Yrs': u'  23.6%', 'numUndergrad': u'14,327', 'nearestBus': u'1 mile(s) from campus in Richmond', 'otherCost': [u'$2,700'], 'numFaculty': u'Not reported', 'areaPop': u'32,550', 'campusSize': u'500 acres', 'download_slot': 'www.collegedata.com', 'coed': u'Yes', 'pctAdvancedStudy': u'Not reported', 'undergradMen': u'8,181 (57.1%)', 'download_timeout': 180.0, 'avgAge': u'23', 'SATReading': u'481 average ', 'gotFinancialNeed': u'7,775 (97.6%) of applicants with financial need', 'fraternities': u'   8% of men participate', 'avgIndebtness': u' $34,358', 'mascot': u'Colonel', 'bookCost': [u'$1,000'], 'avgAward': u'$10,962', 'admissionRate': u'71% of 10,215 applicants were admitted', 'name': u'Eastern Kentucky University', 'SATWriting': u'457 average ', 'url': u'https://www.collegedata.com/cs/data/college/college_pg01_tmpl.jhtml?schoolId=944', 'institutionType': u'Public', 'pctEmployedin6Mo': u'Not reported', 'depth': 6, 'download_latency': 2.5333781242370605, 'undergradWomen': u'8,181 (57.1%)'}


### if y is missing, drop the row. if many of the x values for that obs is missing, drop it. if one x is missing, and the rest of the column is fairly well populated, fill with local/global median/mean

In [860]:
#When scraping the cost variables, I used .extract() rather than .extract_first() because some of the total/tuition
#costs had in state and out-of-state values. I wasn't sure if this was also the case for the other costs so wanted
#to explore. .extract() returns a list of values, and since I have decided to use the out-of-state cost for now, this
#reassigns totalCost and tuitionCost to the out-of-state values for schools which had them. If they only had one value,
#the cost remained the same.
df['totalCost'][df.totalCost.str.len() == 2] = pd.DataFrame(df.loc[df.totalCost.str.len() == 2].totalCost.values.tolist(), index= df.loc[df.totalCost.str.len() == 2].index)[1]
df['tuitionCost'][df.tuitionCost.str.len() == 2] = pd.DataFrame(df.loc[df.tuitionCost.str.len() == 2].tuitionCost.values.tolist(), index= df.loc[df.tuitionCost.str.len() == 2].index)[1]

In [861]:
#This code brings the cost variables out of the list format and removes all non numeric characters
costColumns = ['totalCost', 'tuitionCost', 'bookCost', 'roomCost', 'otherCost']

for column in costColumns:
    df[column] = df[column].apply(lambda x: ''.join(x))
    df[column] = df[column].apply(lambda x: ''.join([c for c in x if c.isnumeric()]))

In [862]:
#If the observation does not have a dependent variable, drop that record
df.totalCost = df.totalCost.replace(["\u00a0", "Not reported", 'Not available', ''], np.nan)
df = df[df['totalCost'].notnull()]
df.totalCost = pd.to_numeric(df.totalCost)

In [863]:
#the class size variable is brought in as a list of bins with the % each bin is represented
#e.g. 2-9 students: 39% of classes, 10-19 students: 31% of classes etc.
#this code is used to take the median of each bin, multiply it by the % its represented, and then summing all of 
#these values to calculate the average class size for each college
classSizedf = pd.DataFrame(df.classSize.values.tolist(), index= df.index)
#replace Nones and "Not reported" with NaN
classSizedf = classSizedf.replace("Not reported", np.nan)
classSizedf.fillna(value=np.nan, inplace = True)
#cleaning white space at beginning and end of lists, and splitting into elements
classSizedf = classSizedf.apply(lambda x: x.str.strip())
classSizedf = classSizedf.apply(lambda x: x.str.split(' '))

#for most of the lists, the first element will be the bin (e.g. 2-9 students), and the third element will be the %.
#these lists have a length of 5. The other lists be in the format [Over 100 students: 0% of classes] and have a 
#length of 6, so we can just get the second element and multiply it by the 4th.
def getMedian(l):
    if len(l) == 6:
        return float(l[1].strip()) * float(l[3].strip('%'))/100
    else:
        return sum(map(float, l[0].split('-')))/2 * float(l[2].strip('%'))/100

classSizeBins = classSizedf.applymap(lambda x: x if type(pd.isnull(x)) == bool else getMedian(x))
classSizeBins['avgClassSize'] = (classSizeBins[0].fillna(0) + classSizeBins[1].fillna(0) + 
                                 classSizeBins[2].fillna(0) + classSizeBins[3].fillna(0) +
                                 classSizeBins[4].fillna(0) + classSizeBins[5].fillna(0) +
                                 classSizeBins[6].fillna(0))

df['avgClassSize'] = classSizeBins['avgClassSize']
df.avgClassSize.replace(0, np.nan, inplace = True)


In [864]:
#replacing non breaking spaces, blank strings, and Not Reported/available with NaN
#doing this again after I have dealt with all variables that were lists
df = df.replace(["\u00a0", "Not reported", 'Not available', ''], np.nan)
df.fillna(value=np.nan, inplace = True)

In [865]:
#check how many nulls are within each column. There are some columns which have 40%+ nulls, and so I have decided to
#drop these variables, because it is hard to impute when there are such a large amount missing. undergradMen should
#also be the complement of undergradWomen, and coed flag should be very highly correlated with the % of undergradWomen,
#so dropping these variables.
#dropping SATWriting because not many schools require this anymore, so there are a lot of null values, and 
#tuition, book, room, and other cost are components of total cost, so I am removing these variables too.

df = df.drop(['averageSalary','sororities', 'pctEmployedin6Mo', 'pctAdvancedStudy', 'numFaculty',
              'fraternities', 'coed', 'undergradMen', 'tuitionCost', 'bookCost', 'roomCost',
              'otherCost', 'SATWriting'], axis = 1)


In [866]:
#Private for-profit schools have a vastly different structure than private/public colleges, and since the number
#of observations with an institution type of "Private for-profit" is only 11, I am dropping these observations.

df = df[df.institutionType != "Private for-profit"]

In [867]:
#for values with a range, take the average, else just return the value
def getScore(s):
    s = s.split(' ')[0]
    if '-' in s:
        return sum(map(float, s.split('-')))/2
    else:
        return float(s)

df.ACT = df.ACT.apply(lambda x: x if pd.isnull(x) else getScore(x))


In [868]:
df.SATMath = df.SATMath.apply(lambda x: x if pd.isnull(x) else getScore(x))
df.SATReading = df.SATReading.apply(lambda x: x if pd.isnull(x) else getScore(x))


In [869]:
#SAT Scores investigation
df[['SATMath', 'SATReading']].describe()
df[df.SATReading > df.SATReading.quantile(0.95)][['url', 'SATReading']].sort_values(by = "SATReading", ascending = False)
# average should be 465 for college at index 1610 (incorrectly stated on website, correct value is underneath)
df.SATReading.loc[1610,] = 465
#checked values that seemed like outliers, and imputed based on what the value should be

df.SATMath.loc[633,] = 750
df.SATMath.loc[370,] = 505
df.SATMath.loc[[1718,1760],] = np.nan
df.SATReading.loc[[435,1760],] = np.nan
df.SATReading.loc[1787, ] = 570
df.ACT.loc[435, ] = 22.5
df.ACT.loc[1789, ] = 21.5
df.ACT.loc[1413, ] = 23
df.avgGPA.loc[176,] = np.nan

In [870]:
#found two rows where the areaPop was pulling the incorrect information, set to nan.
df[df.areaPop.apply(lambda x: False if pd.isnull(x) else x.isalpha())]
df.areaPop.loc[1271,] = np.nan
df.areaPop = df.areaPop.apply(lambda x: x if pd.isnull(x) else float(''.join([c for c in x if c.isnumeric()])))
#found an outlier in town of Ypsilanti, Michigan. reassigned the area population to 2017 census value
df[df.areaPop > df.areaPop.quantile(0.90)][['url', 'areaPop']].sort_values(by = "areaPop", ascending = False)
df.areaPop.loc[1858,] = 21018

In [871]:
df.avgAge = pd.to_numeric(df.avgAge)
df.avgAward = df.avgAward.apply(lambda x: x if pd.isnull(x) else float(''.join([c for c in x if c.isnumeric()])))
df.avgGPA = df.avgGPA.apply(lambda x: x if pd.isnull(x) else float(x.split(' ')[0]))
df.avgIndebtness = df.avgIndebtness.apply(lambda x: x if pd.isnull(x) else float(''.join([c for c in x if c.isnumeric()])))
df.campusSize = df.campusSize.apply(lambda x: x if pd.isnull(x) else float(''.join([c for c in x if c.isnumeric()])))
df.numGrad = df.numGrad.apply(lambda x: x if pd.isnull(x) else float(''.join([c for c in x if c.isnumeric()])))
df.numUndergrad = df.numUndergrad.apply(lambda x: x if pd.isnull(x) else float(''.join([c for c in x if c.isnumeric()])))
df.pctInternationalStudents = df.pctInternationalStudents.apply(lambda x: x if pd.isnull(x) else float(x.split(' ')[0].strip('%'))/100)
df.pctStudentsGraduatingin4Yrs = df.pctStudentsGraduatingin4Yrs.apply(lambda x: x if pd.isnull(x) else float(x.strip('%'))/100)
df.pctWithLoans = df.pctWithLoans.apply(lambda x: x if pd.isnull(x) else float(x.strip('%'))/100)
df.undergradWomen = df.undergradWomen.apply(lambda x: x if pd.isnull(x) else float(''.join(c for c in x if c not in '()').split(' ')[1].strip('%'))/100)
df.admissionRate = df.admissionRate.apply(lambda x: x if pd.isnull(x) else float(x.split('%')[0])/100)
df.institutionType = df.institutionType.apply(lambda x: x if pd.isnull(x) else 0 if x == "Public" else 1)

In [872]:
#schoolid 763 and 682, 1670 got financial need % should be NaNs, since they have no denominator
#row index of 86, 199, 1367
df.gotFinancialNeed.loc[[86,199,1367]] = np.nan
df.gotFinancialNeed = df.gotFinancialNeed.apply(lambda x: x if pd.isnull(x) else x.split(' ')[1])

#school id 1486, 886, 187, 1125, 911, 652, 1308, 861, 888, 804 need to manually impute (they just didnt report the 
#% in the field that I scraped. I went back to the website and manually calculated the % for the list below.
#these schools have the row index of 413, 929, 979, 1090, 1115, 1338, 1407, 1505, 1591,1845
indexList = [413, 929, 979, 1090, 1115, 1338, 1407, 1505, 1591, 1845]
valueList = ["(99.7%)", "(99.4%)", "(99.6%)", "(99.8%)", "(96.3%)", "(98.2%)", 
               "(96.6%)", "(99.8%)", "(96.0%)", "(95.3%)"]

for i in range(len(indexList)):
    df.gotFinancialNeed.loc[indexList[i]] = valueList[i]

# #All the other school IDs that did not have a % were in fact 100% after taking a look at the website
df.gotFinancialNeed.loc[[34,166,181,220,322,506,711,938,1206,1314,1498,1574,1596,1655,1714,1947]] = "(100.0%)"
df.gotFinancialNeed = df.gotFinancialNeed.apply(lambda x: x if pd.isnull(x) else float(''.join(c for c in x if c not in '()').strip('%'))/100)

In [873]:
#priority housing should realistically mean that you get guaranteed housing barring any major issues, so 
#decided to group them together.
df.freshmanHousingGuaranteed = df.freshmanHousingGuaranteed.apply(lambda x: x if pd.isnull(x) else 0 if x == "Freshmen are not guaranteed housing" else 1)


In [691]:
#STILL TO CLEAN - on Hold.
#sportsdiv for now - categorical 
#nearestAirport/bus/Train

#nearestTransportation cleaning LATER

#df.nearestAirport.apply(lambda x: x if pd.isnull(x) else x[3:10]).value_counts()
#df.nearestAirport.apply(lambda x: False if pd.isnull(x) else x.isalpha())
#df.nearestAirport[df.nearestAirport.apply(lambda x: False if pd.isnull(x) else x.split(' ')[0].isalpha())]

### Creating dataset where the NaNs are filled with the mean of the column, by the institutionType

In [888]:
df2 = df.drop(['mascot','nearestAirport', 'nearestBus', 'nearestTrain', 'schoolColors', 
                'sportsDivision', 'classSize'], axis = 1)

In [896]:
df2 = df2.groupby('institutionType', as_index = False).apply(lambda x: x.fillna(x.mean()))

In [877]:
#PICKLING DF TO USE FOR REGRESSION MODELS
pickle.dump(df,open('/Users/kstern/ds/metis/Kevin/Projects/Luther/data/collegeData.pickle','wb'))

In [898]:
#PICKLING DF where NaNs are filled with means
pickle.dump(df2,open('/Users/kstern/ds/metis/Kevin/Projects/Luther/data/collegeData2.pickle','wb'))